# N-gramas

Um **n-grama** é uma sequência contígua de n elementos de um determinado texto. Esses elementos podem ser fonemas, sílabas, palavras ou tokens.

N-gramas podem ser úteis na atividade de prever a n-ésima palavra se baseando nas n-1 palavras anteriores. 

In [ ]:
# Importando dependências necessárias
import re
from nltk import word_tokenize

In [ ]:
# Definindo função de extração de n-gramas
# Veja que n é um parâmetro da função
def ext_n_gram(sentenca, n):
  tokens = word_tokenize(sentenca)
  for i in range(len(tokens)-n+1):
    print(tokens[i:i+n])

In [ ]:
# Testando para 2-gramas
ext_n_gram("The dog and the cat are playing togheter.", 2)

['The', 'dog']
['dog', 'and']
['and', 'the']
['the', 'cat']
['cat', 'are']
['are', 'playing']
['playing', 'togheter']
['togheter', '.']


In [ ]:
# Testando para 3-gramas
ext_n_gram("The dog and the cat are playing togheter.", 3)

['The', 'dog', 'and']
['dog', 'and', 'the']
['and', 'the', 'cat']
['the', 'cat', 'are']
['cat', 'are', 'playing']
['are', 'playing', 'togheter']
['playing', 'togheter', '.']


In [ ]:
# Vamos agora usar o nltk ao invés de fazer a separação à mão
from nltk import ngrams

In [ ]:
# Uso da função importada
list(ngrams('The dog and the cat are playing togheter.'.split(), 2))

[('The', 'dog'),
 ('dog', 'and'),
 ('and', 'the'),
 ('the', 'cat'),
 ('cat', 'are'),
 ('are', 'playing'),
 ('playing', 'togheter.')]

In [ ]:
# Vamos usar uma outra biblioteca com função semelhante a da nltk, a textbloob
!pip install -U textblob
from textblob import TextBlob

In [ ]:
# Frase que usaremos
blob = TextBlob("The dog and the cat are playing togheter.")

In [ ]:
# Usando a textblob para 2-gramas
blob.ngrams(n=2)

[WordList(['The', 'dog']),
 WordList(['dog', 'and']),
 WordList(['and', 'the']),
 WordList(['the', 'cat']),
 WordList(['cat', 'are']),
 WordList(['are', 'playing']),
 WordList(['playing', 'togheter'])]

In [ ]:
# Usando a textblob para 3-gramas
blob.ngrams(n=3)

[WordList(['The', 'dog', 'and']),
 WordList(['dog', 'and', 'the']),
 WordList(['and', 'the', 'cat']),
 WordList(['the', 'cat', 'are']),
 WordList(['cat', 'are', 'playing']),
 WordList(['are', 'playing', 'togheter'])]

# Modelos de linguagem com n-gramas

Como estavamos dizendo, n-gramas podem ser úteis para prever a n-ésima palavra baseando-se apenas na n-1 palavras anteriores. Modelos usados para prever uma determinada sequência de palavras são chamados de **modelos de linguagem**. 

O objetivo principal dos modelos de linguagem é atribuir uma probabilidade a uma determinada sentença ou sequência de palavras.

## 1. Funções que usaremos 

In [ ]:
# Vamos começar importando as dependências necessárias

# Expressões regulares
import re

# Contador de frequência de aparições em um iterable
from collections import Counter

# Trabalhar com partes de um iterable
from itertools import islice

# Capacidade de trabalhar com geradores pseudo-aleatórios
import random

In [ ]:
# Definindo a regex que usaremos para encontrar palavras
regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+"

In [ ]:
# Definindo função para encontrar tokens de um arquivo
def get_tokens(nome_arq):
  with open(nome_arq, 'r') as documento:
    # Pegando vetor com todas as linhas do arquivo
    conteudo = documento.read()
    # Colocando todas as letras do arquivo em minúsculo
    conteudo = conteudo.lower()
  tokens = re.findall(regex, conteudo)
  return tokens

In [ ]:
# Definindo função que pega tokens que não sejam stopwords
def get_tokens_s_stopwords(nome_arq, stopwords = "/content/stopwords_pt.txt"):
  # Pegando o texto
  with open(nome_arq, 'r') as documento:
    conteudo = documento.read()
    conteudo = conteudo.lower()
  # Pegando as stopwords
  with open(stopwords, 'r') as stopwords_arq:
    stopwords = set([])
    for s in stopwords_arq.readline():
      stopwords.add(s.strip().lower())
  # Removendo as stopwords
  tokens = [palavra for palavra in re.findall(regex,conteudo) if w not in stopwords]
  return tokens

In [ ]:
# Função de janela deslizante que retorna uma janela de tamanho n
# A cada chamada dessa função ela se move para o próxima elemento, por isso deslizante
def janela(seq, n=2):
  # Define iter como iterável sobre seq
  it = iter(seq)
  # O resultado é uma tupla que possui a primeira janela de tamanho n sobre iter
  resultado = tuple(islice(it,n))
  # Verifica se existem elementos suficientes para construir a primeira janela de tamanho n
  if len(resultado) == n:
    yield resultado
  # É aqui onde a ação de deslizar a janela ocorre, atualizando resultado com o próximo elemento
  for elem in it:
    resultado = resultado[1:] + (elem,)
    yield resultado


In [ ]:
# Definição da função que retorna a frequência de bigramas e unigramas
def freq_ngramas(palavras):
  unigramas = Counter(palavras)
  bigramas = Counter(janela(palavras, 2))
  return (unigramas, bigramas)

In [ ]:
# Função lambda para calcular as probabilidades baseando-se nas contagens
# A probabilidade a ser calculada é a p(w2) dada uma palavra predecessora w1
# Aqui, unigrmas e bigramas referem-se aos objetos criados na função freq_ngramas
prob_bigram = lambda p1,p2: bigramas[(p1, p2)]/ unigramas[p1]

In [ ]:
# Função que pega uma série de frases e aplica um score de probabilidade delas serem escolhidas
def score(frases):
  for frase in frases:
    palavras = re.findall(regex, frase)
    p = float(1.0)
    for p1, p2 in janela(palavras, 2):
      p = p * prob_bigram(p1,p2)
    print(f"Frase '{frase}' possui p = {p:.20f}")

## 2. Testando o modelo criado

In [ ]:
# Sentenças que usaremos para testar o modelo que desenvolvemos com as funções
sentencas = ["uma noite destas, vindo da cidade",
             "uma noite destas, vindo da", 
             "uma noite destas, vindo", 
             "uma noite destas", 
             "uma noite"]

In [ ]:
# Vamos calcular a probabilidade de cada sentença, nos baseando num livro de Machado de Assis
tokens = get_tokens("/content/dom_casmurro.txt")

In [ ]:
# Formação de unigramas e bigramas usando os tokens capturados
unigramas, bigramas = freq_ngramas(tokens)

In [ ]:
# Verificando os scores de cada sentença
score(sentencas)

Frase 'uma noite destas, vindo da cidade' possui p = 0.00000005316696327186
Frase 'uma noite destas, vindo da' possui p = 0.00000477743141400026
Frase 'uma noite destas, vindo' possui p = 0.00004299688272600236
Frase 'uma noite destas' possui p = 0.00021498441363001179
Frase 'uma noite' possui p = 0.00902934537246049586


In [ ]:
# Vendo a quantidade de tokens, unigramas e bigramas
print(len(tokens))
print(len(unigramas))
print(len(bigramas))

68895
10352
44495


## 3. Usando bigramas para descobrir próxima palavra

In [ ]:
# Definindo função que indica as mais provavéis próxima palavra para uma palavra predecessora
def prob_prox(frase, n=5):
  # Quebra da frase em tokens
  tokens = re.findall(regex, frase)
  # Calcula as probabilidades de todas as palavras em que o bigrama começa por w1 e armazena em uma lista
  probs = {w2 : prob_bigram(w1, w2) for (w1, w2) in bigramas.keys() if w1 == tokens[-1]}
  # Ordenamos e mostramos os n mais prováveis
  for palavra, probabilidade in islice(sorted(probs.items(), 
                                       key = lambda item: item[1], 
                                       reverse = True), n):
    print(f'{frase} + {palavra} -> probabilidade = {probabilidade*100:.2f}%')

In [ ]:
# Vamos testar a função definida acima com o corpus de dom casmurro
tokens = get_tokens('/content/dom_casmurro.txt')
unigramas, bigramas = freq_ngramas(tokens)

In [ ]:
# Testando com um input do usuário
frase = input("\nDigite uma frase: ")
prob_prox(frase)


Digite uma frase: quando
quando + eu -> probabilidade = 10.81%
quando + me -> probabilidade = 4.86%
quando + elle -> probabilidade = 4.86%
quando + lhe -> probabilidade = 3.78%
quando + a -> probabilidade = 3.78%


## 4. Criando classe NGramas

Vamos criar uma classe NGramas que incorpore as funções de modelagem que desenvolvemos até agora.

In [ ]:
class NGramas(object):

  # O construtor da classe aceita um número máximo para n e o objeto com as palavras
  def __init__(self, max_n, palavras = None):
    self.max_n   = max_n
    # Lembrando que Counter() é uma collection, subclasse de dicionário, usada para contar elementos que podem ser representados por hash
    self.contador  = Counter()
    if palavras is not None:
      self.update(palavras)

  # Método que, baseado nas palavras, encontra e conta os unigramas, bigramas, ..., n-gramas
  def update(self, palavras):
    # Realizando a contagem para cada valor de n usando a função janela
    for i in range(1,self.max_n+1):
        self.contador.update(janela(palavras,i))
    # Abaixo temos um caso especial onde a tupla de janela retorna vazia
    self.contador[()] += len(palavras)

  # Calcula as probabilidades para a frase usando a lista de tokens/palavras
  # Usa estratégia recursiva que vai até o 1-grama para ir retornando até os n-gramas
  def probabilidades(self, palavras):
      # Testando se existem palavras para compor o ngrama
      if len(palavras) <= self.max_n:
          return self.probabilidades(palavras)
      else:
          P = 1
          for i in range(len(palavras) - self.max_n + 1):
              ngram = palavras[i:i + self.max_n]
              P     = P * self.probabilidade(ngram)
          return P
  
  # Calcula a probabilidade de determinado n-grama usando seu prefixo
  def probabilidade(self, ngram):
      # Cria ngrama como tupla 
      ngram = tuple(ngram)
      ngram_count = self.Counts[ngram]
      prefix_count = self.Counts[ngram[:-1]]
      # Se uma tupla (n-grama) nao for observada devolvemos zero
      if ngram_count and prefix_count:
          return ngram_count / prefix_count
      else:
          return 0.0
      # Geracao de frases de 'n_words'
  def generate(self, n_words, threshold = random.random()):
      
      # cria uma lista de unigrams
      unigrams = [ngram for ngram in self.Counts.keys() if len(ngram) == 1]
      # Tentamos gerar frases 
      words = []
      
      while len(words) < n_words:
          # o prefixo para o proximo n-grama
          if self.max_n == 1:
              prefix = ()
          else:
              prefix = tuple(words[-self.max_n + 1:])
         
          total     = 0.0
          random.shuffle(unigrams)
          for unigram in unigrams:
              total += self._probability(prefix + unigram)
              if total >= threshold:
                  words.extend(unigram)
                  break
          
          # Se nao for possivel criar uma frase  
          if total == 0.0:
              raise RuntimeError('impossible sequence')
      return(words)